In [30]:
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import os

import sys 
sys.path.insert(0, "../src")
from BrusselsMap import BrusselsMap
from GeoJsonHandler import GeoJsonHandler
from OSMGeoJsonHandler import OSMGeoJsonHandler

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 500)

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
r_path = r"../data/osm/overpass-turbo/office.geojson"
path = os.path.join(os.getcwd(), r_path)

work = OSMGeoJsonHandler(r_path, feature_type='office')

In [24]:
work.gdf

,ID,TYPE,NAME,geometry,AREA,LAT,LON
0,86778,relation,Orange Belgium SA,"POLYGON ((4.41371 50.87633, 4.41401 50.87599, ...",0.004747,50.876118,4.414414
1,1628306,relation,Graaf de Ferrarisgebouw,"POLYGON ((4.35529 50.85897, 4.35531 50.85897, ...",0.004142,50.859198,4.355898
2,2026215,relation,Mission of the United States of America to the...,"MULTIPOLYGON (((4.36726 50.84423, 4.36682 50.8...",0.001733,50.844092,4.366926
3,2982811,relation,Fédérale Assurance - Federale Verzekering,"POLYGON ((4.35153 50.84555, 4.35167 50.84543, ...",0.001187,50.845643,4.351615
4,3000389,relation,Paleis der Natie (Belgisch federaal parlement)...,"POLYGON ((4.36442 50.84675, 4.36449 50.84689, ...",0.018697,50.846877,4.364645
...,...,...,...,...,...,...,...
1089,9249091398,node,Growfunding,POINT (4.35014 50.86428),0.000332,50.864281,4.350137
1090,9256387753,node,Regus,POINT (4.36008 50.84658),0.000332,50.846579,4.360083
1091,9267641994,node,FORATOM,POINT (4.36667 50.84097),0.000332,50.840973,4.366669
1092,9270130817,node,Eliane,POINT (4.29875 50.81355),0.000332,50.813552,4.298745


## Assign neighborhoods

In [25]:
#Load geodata
r_path = r"../data/final_data/sector.json"
path = os.path.join(os.getcwd(), r_path)
sectors = GeoJsonHandler(path, "NAME_FRE")

In [26]:
work.assign_ngh(sectors)

  0%|                                                                                                                                 | 0/1094 [00:00<?, ?it/s]C:\Users\vic_t\Anaconda3\envs\mfe_env\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1094/1094 [01:29<00:00, 12.24it/s]


In [28]:
work.gdf['AMENITY'] = 'office'
work.gdf = work.gdf[['ID', 'TYPE', 'NAME', 'AMENITY', 'geometry', 'AREA', 'LAT', 'LON', 'NAME_FRE']]
work.gdf.head()

,ID,TYPE,NAME,AMENITY,geometry,AREA,LAT,LON,NAME_FRE
0,86778,relation,Orange Belgium SA,office,"POLYGON ((4.41371 50.87633, 4.41401 50.87599, ...",0.004747,50.876118,4.414414,INDUSTRIE OTAN
1,1628306,relation,Graaf de Ferrarisgebouw,office,"POLYGON ((4.35529 50.85897, 4.35531 50.85897, ...",0.004142,50.859198,4.355898,QUARTIER NORD
2,2026215,relation,Mission of the United States of America to the...,office,"MULTIPOLYGON (((4.36726 50.84423, 4.36682 50.8...",0.001733,50.844092,4.366926,QUARTIER ROYAL
3,2982811,relation,Fédérale Assurance - Federale Verzekering,office,"POLYGON ((4.35153 50.84555, 4.35167 50.84543, ...",0.001187,50.845643,4.351615,GRAND PLACE
4,3000389,relation,Paleis der Natie (Belgisch federaal parlement)...,office,"POLYGON ((4.36442 50.84675, 4.36449 50.84689, ...",0.018697,50.846877,4.364645,QUARTIER ROYAL


In [29]:
## Save results
r_path = r"../data/final_data/osm/office.csv"
path = os.path.join(os.getcwd(), r_path)

work.gdf.to_csv(path, sep=',', encoding='utf-8-sig', index=False, chunksize=10000)